In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
%matplotlib inline
import numpy as np
import os, shutil, glob, sys, math, cv2, re

import albumentations as albu
import pandas as pd
import seaborn as sns

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
# from torch.utils import load_state_dict_from_url
from torch.utils.model_zoo import load_url as load_state_dict_from_url
from torch import Tensor
from torch.jit.annotations import List
from torchsummary import summary
from torchvision import models

from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [ ]:
DATA_FOLDER = "/home/Tsung/pathology/data/tcga/"

# model

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        x = x.view(x.size()[0], -1)
        return x

In [ ]:
from typing import Optional, Any

class TransformerEncoderLayer(nn.Module):
    r"""TransformerEncoderLayer is made up of self-attn and feedforward network.
    This standard encoder layer is based on the paper "Attention Is All You Need".
    Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez,
    Lukasz Kaiser, and Illia Polosukhin. 2017. Attention is all you need. In Advances in
    Neural Information Processing Systems, pages 6000-6010. Users may modify or implement
    in a different way during application.
    Args:
        d_model: the number of expected features in the input (required).
        nhead: the number of heads in the multiheadattention models (required).
        dim_feedforward: the dimension of the feedforward network model (default=2048).
        dropout: the dropout value (default=0.1).
        activation: the activation function of intermediate layer, relu or gelu (default=relu).
    Examples::
        >>> encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        >>> src = torch.rand(10, 32, 512)
        >>> out = encoder_layer(src)
    """

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)

    def __setstate__(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super(TransformerEncoderLayer, self).__setstate__(state)

    def forward(self, src: Tensor, src_mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        r"""Pass the input through the encoder layer.
        Args:
            src: the sequence to the encoder layer (required).
            src_mask: the mask for the src sequence (optional).
            src_key_padding_mask: the mask for the src keys per batch (optional).
        Shape:
            see the docs in Transformer class.
        """
        src = self.norm1(src)
        src2 = self.self_attn(src, src, src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm2(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout2(src2)
        return src

def _get_activation_fn(activation):
    if activation == "relu":
        return F.relu
    elif activation == "gelu":
        return F.gelu

    raise RuntimeError("activation should be relu/gelu, not {}".format(activation))

In [ ]:
# 2D model
class attentionPart(nn.Module):
    def __init__(self, d_model = 512):
        super().__init__()
#         self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=8)
        self.transformer_encoder_layer = TransformerEncoderLayer(d_model=d_model, nhead=8)

        self.attention_layer = nn.Sequential(
            nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=6),
            nn.AdaptiveMaxPool1d(1),
            Flatten(),
        )
       
    def forward(self, x):       
        attention = self.attention_layer(x)
        return attention

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
# init 一定要做
class other_feature_encoding(nn.Module):
    def __init__(self, num_features = 100, d_model = 512):
        super().__init__()
        self.n_features = num_features
        self.wm = nn.Parameter(torch.Tensor(num_features, d_model))
        self.wm = nn.init.normal_(self.wm)
    def forward(self,x):
        x = x.view(-1, self.n_features, 1)
        x = x * self.wm
        return x
    

In [ ]:
from encoders.resnet import *

In [ ]:
# 2D model
class classifier(nn.Module):
    def __init__(self, max_images = 200, d_model = 512, number_other_feature = 200, dropout=0.1, nclass=10):
        super().__init__()
        self.nclass = nclass
        self.CNN = resnetSmall()
        self.CNN.fc = nn.Identity()
        self.pos_encoder = PositionalEncoding(d_model=d_model, max_len=max_images)
        
        self.other_feature_encoding = other_feature_encoding(num_features = number_other_feature, d_model = d_model)
    
        self.attention_layer = attentionPart(d_model)
        
        self.classify = nn.Sequential(
            nn.Linear(max_images + number_other_feature, 128),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(128, 32),
            nn.ReLU(True),
            nn.Dropout(),    
            nn.Linear(32, nclass)
        )
        # softmax
    def forward(self, x, x2):
        batch_size, timesteps, C, H, W = x.size()
        cnn_input = x.view(batch_size * timesteps, C, H, W)
        cnn_output = self.CNN(cnn_input)
        other_feature = self.other_feature_encoding(x2)
        att_input = cnn_output.view(batch_size, timesteps, -1)
        att_input = self.pos_encoder(att_input)
        # concat two data
        att_input = torch.cat([att_input, other_feature], dim = 1)
        att_output = self.attention_layer(att_input)
        cls = self.classify(att_output)
        return cls

In [ ]:
# model = classifier(max_images = 10, d_model = 512, number_other_feature=17, nclass = 3)

In [ ]:
# a = torch.randn(1, 10, 3, 512, 512)
# b = torch.randn(1, 17)
# out = model(a, b)
# out = out.detach().cpu().numpy()
# print(out.shape)

# data

In [ ]:
def cvt2Classy(label):
    cut = np.array([100,200])
    
    _foo = (label < cut)*1
    if np.sum(_foo) == 0:
        return len(cut)
    else:
        for _temp, _i in enumerate(_foo):
            if _i == 1:
                return _temp

In [ ]:
# 數量500多，有些沒有images
import pandas as pd
df = pd.read_csv('./data/coad_Mutation_Count.txt', delimiter="\t")
cohort_count_dict = {row[1]: row[3] for index,row in df.iterrows()}

In [ ]:
import pickle
with open('./no_RNA_no_onehot_other_feature.pkl', 'rb') as fp:
    cohort_other_feature_dict = pickle.load(fp)

# using percentile, only choose high and low

In [ ]:
def high_and_low_cohort(all_cohorts = None, cohort_mc_dict = None ):
    # all cohort 只有300多人， cohort_mc_dict 有 500 多人
    all_counts = np.array(list(cohort_mc_dict.values()))
    percentile = np.percentile(all_counts, [25,50,75])
    
    low_cohort = []
    high_cohort = []
    for one_cohort in all_cohorts:
        if cohort_mc_dict[one_cohort] <= percentile[0]:
            low_cohort.append(one_cohort)
        elif cohort_mc_dict[one_cohort] >= percentile[2]:
            high_cohort.append(one_cohort)
    print("high-{} : {}\nlow-{} : {}".format(percentile[2], len(high_cohort), percentile[0], len(low_cohort)))
    return np.array(high_cohort), np.array(low_cohort)
high_cohort, low_cohort = high_and_low_cohort(all_cohorts = list(cohort_other_feature_dict.keys()), cohort_mc_dict = cohort_count_dict)

using_cohorts = np.concatenate((high_cohort, low_cohort))
print(len(using_cohorts))

In [ ]:
train_cohorts, valid_cohorts = train_test_split(
    using_cohorts, test_size=0.33, random_state=0)
print(len(train_cohorts), len(valid_cohorts))

In [ ]:
dense200_folders = os.listdir(os.path.join(DATA_FOLDER, '1000dense200_npy'))
train_image_folders = []
valid_image_folders = []
train_counts = []
valid_counts = []

for folder_name in dense200_folders:
    if not folder_name.startswith('TCGA'):
        continue
    cohort_name = folder_name[:12]
    flag = folder_name[13]
    if flag == '1':
        continue
    if cohort_name in train_cohorts:
        train_image_folders.append(folder_name)
        if cohort_name in high_cohort:
            train_counts.append(1)
        elif cohort_name in low_cohort:
            train_counts.append(0)
    if cohort_name in valid_cohorts:
        valid_image_folders.append(folder_name)
        if cohort_name in high_cohort:
            valid_counts.append(1)
        elif cohort_name in low_cohort:
            valid_counts.append(0)
        
train_image_folders = np.array(train_image_folders)
valid_image_folders = np.array(valid_image_folders)
train_counts = np.array(train_counts)
valid_counts = np.array(valid_counts)
print(len(train_image_folders), len(valid_image_folders), len(train_counts), len(valid_counts))

# using 100,200,300, low, medium, high

In [ ]:
train_cohorts, valid_cohorts = train_test_split(
    list(cohort_other_feature_dict.keys()), test_size=0.33, random_state=0)
print(len(train_cohorts), len(valid_cohorts))

In [ ]:
dense200_folders = os.listdir(os.path.join(DATA_FOLDER, '1000dense200'))
train_image_folders = []
valid_image_folders = []
train_counts = []
valid_counts = []

for folder_name in dense200_folders:
    if not folder_name.startswith('TCGA'):
        continue
    cohort_name = folder_name[:12]
    flag = folder_name[13]
    if flag == '1':
        continue
    if cohort_name in train_cohorts:
        train_image_folders.append(folder_name)
        train_counts.append(cvt2Classy(cohort_count_dict[cohort_name]))
    if cohort_name in valid_cohorts:
        valid_image_folders.append(folder_name)
        valid_counts.append(cvt2Classy(cohort_count_dict[cohort_name]))
        
train_image_folders = np.array(train_image_folders)
valid_image_folders = np.array(valid_image_folders)
train_counts = np.array(train_counts)
valid_counts = np.array(valid_counts)

# class weight

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_counts),
                                                 train_counts)
print(class_weights)

In [ ]:
for i in range(3):
    print(np.count_nonzero(train_counts == i))
print('\n')
for i in range(3):
    print(np.count_nonzero(valid_counts == i))

In [ ]:
import seaborn as sns
sns.distplot(train_counts)

In [ ]:
import seaborn as sns
sns.distplot(valid_counts)

# train test dataset

In [ ]:
def get_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),
        albu.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p = 0.5),
        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(test_transform)

def get_validaugmentation(image_size):
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.Resize(height = image_size, width = image_size, always_apply=True),
    ]
    return albu.Compose(test_transform)

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

# https://github.com/pytorch/vision/blob/master/torchvision/transforms/functional.py, to_tensor     
def to0_1(x, **kwargs):
    return x/255

def get_preprocessing():
    _transform = [
        albu.Lambda(image=to_tensor, mask=to_tensor),
        albu.Lambda(image=to0_1, mask=to0_1),
    ]
    return albu.Compose(_transform)

In [ ]:
class Dataset(BaseDataset):
    
    def __init__(self, image_folder_array, label_array, other_feature_dict, image_size = None, augmentation=None, preprocessing=None):
        self.image_folder_array = image_folder_array
        self.label_array = label_array
        self.other_feature_dict = other_feature_dict
        
        self.image_size = image_size
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        fp = self.image_folder_array[i]
        images = np.load(os.path.join(DATA_FOLDER, '1000dense200_npy', fp))
        
        for idx, img in enumerate(images):
            if self.augmentation:
                sample = self.augmentation(image=img)
                images[idx] = sample['image']
        images = images.transpose(0,3,1,2).astype('float32')
            
        label = self.label_array[i]
        label = label.astype('int64')

        other_feature = self.other_feature_dict[fp[:12]].astype('float32')
            
        return images, other_feature, label
        
    def __len__(self):
        return len(self.image_folder_array)

# model settings

In [ ]:
def adjust_learning_rate(optimizer, epoch, init_lr = 1e-4):
    """Sets the learning rate to the initial LR decayed by 10 every 10 epochs"""
    lr = init_lr * (0.1 ** (epoch // 10))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
# def make_weights_for_balanced_classes(_y):
#     nclasses = len(np.unique(_y))
#     weight_per_class = {i: 0. for i in range(10)}
#     count_per_class = {i: 0. for i in np.unique(_y)}
#     for i in _y:
#         count_per_class[i] += 1    
#     for i in np.unique(_y):
#         weight_per_class[i] = len(_y)/float(count_per_class[i])
#     weights = [0.] * len(_y)
#     for idx, val in enumerate(_y):
#         weights[idx] = weight_per_class[val]
#     return weights, weight_per_class

In [ ]:
# weights, weight_per_class = make_weights_for_balanced_classes(np.log(train_counts).astype('uint8'))
# weights = torch.DoubleTensor(weights)
# sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)

# _temp = 0
# for key, value in weight_per_class.items():
#     _temp += value

# weight_per_class = [weight_per_class[i]/_temp for i in range(10)]

In [ ]:
batch_size = 1
nclass = 2
epoch = 50
model_arch = 'resnetSmall'
init_lr = 1e-4
target = 'cls'

nfeature = None
for key, value in cohort_other_feature_dict.items():
    if nfeature is None:
        nfeature = len(value)
    if nfeature != len(value):
        print("Error, feature number error")
print(nfeature)

In [ ]:
train_dataset = Dataset(
    train_image_folders,
    train_counts,
    cohort_other_feature_dict,
    augmentation = get_augmentation()
)

valid_dataset = Dataset(
    valid_image_folders,
    valid_counts,
    cohort_other_feature_dict,
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                              shuffle=True, num_workers=12)
    
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, 
                          shuffle=False, num_workers=12)

In [ ]:
model = classifier(max_images = 200, d_model = 512, number_other_feature=nfeature, nclass = nclass)
model

In [ ]:
import utils
train_loss = 'ce'
train_metric = 'acc'

loss = utils.metrics.CrossEntropy(weight=class_weights.astype('float32'))
metrics = [
    utils.metrics.Accuracy(),
]


optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=init_lr),
])

# train

In [ ]:
import sys
import torch
from tqdm import tqdm as tqdm
# import segmentation_models_pytorch as smp

class Epoch:

    def __init__(self, model, loss, metrics, stage_name, device='cpu', verbose=True):
        self.model = model
        self.loss = loss
        self.metrics = metrics
        self.stage_name = stage_name
        self.verbose = verbose
        self.device = device

        self._to_device()

    def _to_device(self):
        self.model.to(self.device)
        self.loss.to(self.device)
        for metric in self.metrics:
            metric.to(self.device)

    def _format_logs(self, logs):
        str_logs = ['{} - {:.4}'.format(k, v) for k, v in logs.items()]
        s = ', '.join(str_logs)
        return s

    def batch_update(self, x, x2, y):
        raise NotImplementedError

    def on_epoch_start(self):
        pass

    def run(self, dataloader):

        self.on_epoch_start()

        logs = {}
        loss_meter = utils.meter.AverageValueMeter()
        metrics_meters = {metric.__name__: utils.meter.AverageValueMeter() for metric in self.metrics}

        with tqdm(dataloader, desc=self.stage_name, file=sys.stdout, disable=not (self.verbose)) as iterator:
            for x, x2, y in iterator:
                x, x2, y = x.to(self.device), x2.to(self.device), y.to(self.device)
                loss, y_pred = self.batch_update(x, x2, y)

                # update loss logs
                loss_value = loss.cpu().detach().numpy()
                loss_meter.add(loss_value)
                loss_logs = {self.loss.__name__: loss_meter.mean}
                logs.update(loss_logs)

                # update metrics logs
                for metric_fn in self.metrics:
                    metric_value = metric_fn(y_pred, y).cpu().detach().numpy()
                    metrics_meters[metric_fn.__name__].add(metric_value)
                metrics_logs = {k: v.mean for k, v in metrics_meters.items()}
                logs.update(metrics_logs)

                if self.verbose:
                    s = self._format_logs(logs)
                    iterator.set_postfix_str(s)

        return logs


class TrainEpoch(Epoch):

    def __init__(self, model, loss, metrics, optimizer, device='cpu', verbose=True):
        super().__init__(
            model=model,
            loss=loss,
            metrics=metrics,
            stage_name='train',
            device=device,
            verbose=verbose,
        )
        self.optimizer = optimizer

    def on_epoch_start(self):
        self.model.train()

    def batch_update(self, x, x2, y):
        self.optimizer.zero_grad()
        prediction = self.model.forward(x, x2)
        loss = self.loss(prediction, y)
        loss.backward()
        self.optimizer.step()
        return loss, prediction


class ValidEpoch(Epoch):

    def __init__(self, model, loss, metrics, device='cpu', verbose=True):
        super().__init__(
            model=model,
            loss=loss,
            metrics=metrics,
            stage_name='valid',
            device=device,
            verbose=verbose,
        )

    def on_epoch_start(self):
        self.model.eval()

    def batch_update(self, x, x2, y):
        with torch.no_grad():
            prediction = self.model.forward(x, x2)
            loss = self.loss(prediction, y)
        return loss, prediction

In [ ]:
use_cuda = torch.cuda.is_available()
DEVICE = torch.device("cuda:0" if use_cuda else "cpu")

train_epoch = TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
import time
current_time = time.strftime("%Y_%m_%d_%H_%M", time.localtime())

model_name = '/home/Tsung/pathology/weight/' + "{}_MCnt_end2end_image&sideinfo_{}_loss_{}_bs_{}".format(
    current_time, target, train_loss, batch_size)

cur_metric = 0

train_history = []
valid_history = []
for i in range(0, epoch):
    print('\nEpoch: {}, batch: {}'.format(i, batch_size))
    
    # lr_scheduler.step()
    adjust_learning_rate(optimizer, i, init_lr = init_lr)
    for param_group in optimizer.param_groups:
        print(param_group['lr'])
    
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)

    train_history.append(train_logs)
    valid_history.append(valid_logs)
    
    if cur_metric < valid_logs[metrics[0].__name__]:
        cur_metric = valid_logs[metrics[0].__name__]
        torch.save(model.state_dict(), model_name+"_epoch{}_{}:{:.4f}".format(i,train_metric, cur_metric)+".h5")
        print('Model saved!', model_name+"_epoch{}_{}:{:.4f}".format(i,train_metric, cur_metric)+".h5")

#     if cur_metric > valid_logs[metrics[0].__name__]:
#         cur_metric = valid_logs[metrics[0].__name__]
#         torch.save(model.state_dict(), model_name+"_epoch{}_{}:{:.4f}".format(i,train_metric, cur_metric)+".h5")
#         print('Model saved!', model_name+"_epoch{}_{}:{:.4f}".format(i,train_metric, cur_metric)+".h5")

In [ ]:
def plt_history(history):
    loss1 = []
    metric1 = []
    metric2 = []
    for i in history:
        loss1.append(i['cross entropy'])
        metric1.append(i['accuracy'])
    plt.figure()
    p1 = plt.plot(loss1, label='cross entropy')
    p2 = plt.plot(metric1, label='accuracy')
    plt.legend()

In [ ]:
model.eval()

In [ ]:
for data in train_loader:
    img, info, gt = data
    img = img.to(DEVICE)
    info = info.to(DEVICE)
    with torch.no_grad():
        pred = model(img, info)
    pred = torch.softmax(pred, dim = 1)
    pred = torch.argmax(pred, dim = 1)
    pred = pred.detach().cpu().numpy()
    print(pred, gt)

# valid

In [ ]:
train_dataset = Dataset(
    train_cohorts,
    train_counts,
#     augmentation=get_training_augmentation(),
#     preprocessing=get_preprocessing(),
)

valid_dataset = Dataset(
    test_cohorts,
    test_counts,
#     augmentation=get_validation_augmentation(),
#     preprocessing=get_preprocessing(),
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                          shuffle=True, num_workers=batch_size)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, 
                          shuffle=False, num_workers=batch_size)

In [ ]:
model = classifier(max_images = 200, d_model = 512, dropout=0.1)
model.load_state_dict(torch.load('./weight/2020_09_09_16_07_mCount-reg-log-end2end_sampler_loss:model_arch_bs:cetest.h5'))
model = model.cuda()

In [ ]:
gt_list = []
pred_list = []
_temp = 0
metric = Accuracy()
for data in train_loader:
    images, x2, labels = data
    images = images.cuda()
    x2 = x2.cuda()
    labels = labels.cuda()
    with torch.no_grad():
        pred = model.forward(images, x2)
    pred = torch.softmax(pred, axis = 1)
    pred = torch.argmax(pred, axis = 1)
    pred = pred.detach().cpu().numpy()
    pred = pred.squeeze()
    pred_list.extend(pred)

    labels = labels.detach().cpu().numpy()
    gt_list.extend(labels)
gt_list = np.array(gt_list)
pred_list = np.array(pred_list)

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(100,10))
plt.plot(pred_list, color="g")
plt.plot(gt_list, color="r")

In [ ]:
for gt, pred in zip(gt_list, pred_list):
    print("{:.4f}, {:.4f}, {:.4f}".format(gt, pred, np.abs(gt-pred)))

In [ ]:
np.corrcoef(gt_list, pred_list)

whole slide

zStats density

transformer pretrain
svs 加入空間關係

consine scheduler   